# Installing everything needed 

I keep on having an issue with this stage, and I am worried about it for running the model etc. 

In [4]:
import os
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import bertopic

from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from bertopic.representation import KeyBERTInspired  # Correct import

from umap import UMAP
from hdbscan import HDBSCAN
import pycountry

from sklearn.cluster import DBSCAN
from sklearn.metrics.pairwise import cosine_similarity

import time

import re

import plotly.graph_objects as go
import plotly.express as px
from sklearn.metrics.pairwise import cosine_similarity

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler



## 1. Creating CSV

Breaking on every line has had some issues so I will use breaking on every sentence instead. 

In [8]:

root_folder_path = '/Users/student/Desktop/LSE /Year 2/Capstone/NLP Week 11/Sharing with Juan/UN_Speeches'

def parse_filename(filename):
    name_parts = filename.replace('.txt', '').split('_')

    if len(name_parts) < 3:
        return None, None, None  

    country = name_parts[0]
    session = name_parts[1]
    
    try:
        year = int(name_parts[2])
    except ValueError:
        return None, None, None  

    return country, session, year

def preprocess_text(text):
    # Temporarily replace dots in common abbreviations with placeholders
    abbreviation_patterns = [r'\bMs\.', r'\bMr\.', r'\bDr\.', r'\bPresident\.']
    for pattern in abbreviation_patterns:
        text = re.sub(pattern, lambda match: match.group(0).replace('.', '[DOT]'), text)
    return text

def postprocess_sentence(sentence):
    return sentence.replace('[DOT]', '.')

sentence_data = []

for subfolder in os.listdir(root_folder_path):
    subfolder_path = os.path.join(root_folder_path, subfolder)
    
    if os.path.isdir(subfolder_path) and "Session" in subfolder:
        for filename in os.listdir(subfolder_path):
            if not filename.endswith('.txt') or filename.startswith('.'):  
                continue  # Skip hidden/system files like .DS_Store
            
            country, session, year = parse_filename(filename)
            if country is None:  # Skip if parsing failed
                continue  
            
            file_path = os.path.join(subfolder_path, filename)
            with open(file_path, 'r', encoding='utf-8') as file:
                text_content = file.read()
            
            # Preprocess text
            text_content = preprocess_text(text_content)
            text_content = text_content.replace('\n', ' ')  # Replace line breaks with spaces
            
            # Split by full stops while ignoring placeholders for abbreviations
            sentences = re.split(r'(?<!\[DOT])\.\s+', text_content)
            
            for sentence in sentences:
                sentence = sentence.strip()
                if sentence: 
                    sentence_data.append([country, session, year, postprocess_sentence(sentence)])

sentence_df = pd.DataFrame(sentence_data, columns=['Country', 'Session', 'Year', 'Sentence'])
sentence_df.to_csv('un_speeches_split_by_sentence.csv', index=False)

country_code_to_name = {country.alpha_3: country.name for country in pycountry.countries}
sentence_df['Country Name'] = sentence_df['Country'].map(country_code_to_name)





Checking what the df looks like - it seems to have worked well. Splitting on sentences seems to be more robust than splitting on paragraphs. 

In [10]:
sentence_df.to_csv('/Users/student/Desktop/LSE /Year 2/Capstone/NLP Week 11/Sharing with Juan/28th Feb/correct_sentence_df.csv',index = False)

In [11]:
sentence_df = pd.read_csv("correct_sentence_df.csv")

In [12]:
sentence_df.head()

,Country,Session,Year,Sentence,Country Name
0,BRB,73,2018,Let me begin by congratulating Ms. María Ferna...,Barbados
1,BRB,73,2018,"However, I would like to pause at this stage, ...",Barbados
2,BRB,73,2018,Those events include the transit of a tropical...,Barbados
3,BRB,73,2018,Those events are of great concern because the ...,Barbados
4,BRB,73,2018,"I ask myself, what does all that matter? Last ...",Barbados


## 2. 1 Training the BERT model

The main part with this is we would like to be able to keep the BERT model afterwards and save it so it can be reloaded. This is the part that I am struggling with. 

#### Training the BERT model with one-shot embedding
This means that we can train Bert with predefined topics that we are interested in - I have created a list of topics that we care about from before 

In [13]:
zeroshot_topic_list = [
    #Miltiary technology
    "Miltiary Technology",
    "Nuclear Weapons",
    "Biological Weapons",
    "Chemical Weapons",
    #Dual Use Technology
    "Artifical Intelligence",
    "Quantum Technology",
    "Supply chains",
    "Data Security",
    "Biological Engineering",
    "General Emerging Technologies",
    "Atomic Energy",
    "Energy Security",
    "Taiwan Supply Chains",
    "Peaceful development of technology",
    #Civilian technology 
    "Climate change and renewable energy",
    "Electric cars",
    "Financial Technology",
    "Health Technology, Vaccines",
    "Technology in Education",
    #Capacity Building 
    "Digital Divide",
    "Technology Capacity Building",
    "Technology Transfer",
    #Grey Zone
    "Cyberattacks and offensive cyber",
    "Election Interference",
    "Technological Survelliance",
    "Cyber espionage"
]



In [14]:
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# Extract sentences (assuming 'sentence_docs' is a list of sentences)
sentence_docs = sentence_df["Sentence"].tolist()

# Generate embeddings for the sentences
sentence_embeddings = embedding_model.encode(sentence_docs, show_progress_bar=True)


Batches:   0%|          | 0/16323 [00:00<?, ?it/s]

In [15]:
topic_model = BERTopic(
    embedding_model="all-MiniLM-L6-v2", 
    min_topic_size=15,
    zeroshot_topic_list=zeroshot_topic_list,
    zeroshot_min_similarity=.5,
    representation_model=KeyBERTInspired()
)
topics, _ = topic_model.fit_transform(sentence_docs, )

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [17]:
sentence_df['Topic'] = topics

In [54]:
# Assuming `sentence_embeddings` is a 2D numpy array with shape (374959, 384)
sentence_df['Embedding'] = [embedding for embedding in sentence_embeddings]


In [19]:
topic_info = topic_model.get_topic_info()
topic_info.head(26)

,Topic,Count,Name,Representation,Representative_Docs
0,-1,224298,-1_nations_countries_international_africa,"[nations, countries, international, africa, gl...",[In this International Year of Small Island D...
1,0,4253,Nuclear Weapons,"[nuclear, disarmament, treaty, proliferation, ...",[We call upon all nuclear- and non-nuclear-wea...
2,1,80,Biological Weapons,"[biowarfare, weapons, convention, biological, ...",[The need to establish a verification system f...
3,2,480,Chemical Weapons,"[disarmament, chemical, weapons, chemicals, sy...",[I welcome the recent agreement in the Confere...
4,3,25,Artifical Intelligence,"[intelligence, intelligent, ai, smarter, innov...","[Tireless determination, innovation and intell..."
5,4,10,Quantum Technology,"[quantum, advances, leap, technology, developm...",[The quantum leap in transport and communicati...
6,5,45,Supply chains,"[logistics, suppliers, supply, supplies, chain...",[Food security was affected as supply chains w...
7,6,34,Data Security,"[privacy, security, data, safeguarding, secure...","[In this respect, we must reconcile the develo..."
8,7,5,Biological Engineering,"[biotechnology, cloning, genetic, synthetic, e...","[Another key issue for humanity, the future of..."
9,8,109,General Emerging Technologies,"[technologies, technology, technological, deve...","[On the one hand, new technologies are emergin..."


In [20]:
topic_info.shape

(1977, 5)

### 2.2 Save the model

In [21]:
topic_model.save("bertopic_model_2", save_embedding_model=True)

2025-02-28 12:33:33,936 - BERTopic - WARNING: When you use `pickle` to save/load a BERTopic model,please make sure that the environments in which you saveand load the model are **exactly** the same. The version of BERTopic,its dependencies, and python need to remain the same.


### 2.3 Load the pre-saved model 

In [62]:

topic_model = BERTopic.load("bertopic_model")


## 3.1 Visualisation of topics - Seeing top words in the Zeroshot topics 
###  Rename the topics so that they are represented correctly in the various visualisations 

In [43]:
topic_labels = topic_model.generate_topic_labels(nr_words=3,
                                                 topic_prefix=False,
                                                 word_length=10,
                                                 separator=", ")


# Now assign the generated labels to the topics
topic_model.set_topic_labels(topic_labels)


In [45]:
zeroshot_topic_dict = {
    0: "Nuclear Weapons",
    1: "Biological Weapons",
    2: "Chemical Weapons",
    3: "Artificial Intelligence",
    4: "Quantum Technology",
    5: "Supply Chains",
    6: "Data Security",
    7: "Biological Engineering",
    8: "General Emerging Technologies",
    9: "Atomic Energy",
    10: "Energy Security",
    11: "Taiwan Supply Chains",
    12: "Peaceful Development of Technology",
    13: "Climate Change and Renewable Energy",
    14: "Electric Cars",
    15: "Financial Technology",
    16: "Health Technology, Vaccines",
    17: "Technology in Education",
    18: "Digital Divide",
    19: "Technology Capacity Building",
    20: "Technology Transfer",
    21: "Cyberattacks and Offensive Cyber",
    22: "Election Interference",
    23: "Technological Surveillance",
    24: "Cyber Espionage"
}

topic_model.set_topic_labels(zeroshot_topic_dict) 

In [46]:

zeroshot_topic_ids = topic_info[topic_info["Name"].isin(zeroshot_topic_list)]["Topic"].tolist()

## Barchart - showing if topics seem correct 

In [25]:
topic_model.visualize_barchart(topics=zeroshot_topic_ids, custom_labels = True )


## Visualise intertopic distance for zeroshot-topics 

In [26]:
import json

# Generate interactive visualisation
fig = topic_model.visualize_topics(topics=zeroshot_topic_ids)



In [ ]:
topic_vis = topic_model.visualize_topics(
    topics=zeroshot_topic_ids,  # Replace with your topic IDs of interest
    custom_labels=True
)

# Show the visualization
topic_vis.update_layout(
    title="Topic Visualization with Custom Labels",
    title_x=0.5,  # Center the title
    showlegend=True
)

topic_vis.show()

## Visualise connections between topics 

In [28]:
topic_model.visualize_hierarchy(topics = zeroshot_topic_ids, custom_labels = True)

## Heatmap to visualise similarity between topics 

In [29]:
heatmap = topic_model.visualize_heatmap(topics=zeroshot_topic_ids, custom_labels=True)

heatmap

## Visualise the topics over time

In [30]:
topics_over_time = topic_model.topics_over_time(sentence_df['Sentence'], 
                                                sentence_df['Year'],  # Use the existing year column
                                                global_tuning=True, 
                                                evolution_tuning=True,
                                                nr_bins=len(sentence_df['Year'].unique()))  # One bin per year


In [31]:
# Filter the topics_over_time to only include the zero-shot topics
topics_over_time_zeroshot = topics_over_time[topics_over_time['Topic'].isin(zeroshot_topic_ids)]


In [35]:
import plotly.express as px

In [47]:
# Using a cyclic color palette
topic_vis_over_time = topic_model.visualize_topics_over_time(
    topics_over_time=topics_over_time_zeroshot, 
    custom_labels=True
)

topic_vis_over_time


In [50]:
topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=10, custom_labels=True)

In [119]:
fig = topic_model.visualize_topics_over_time(topics_over_time_zeroshot)

# Save the plot as an HTML file
fig.write_html("topics_over_time_zeroshot.html")

In [38]:
fig = topic_model.visualize_topics_over_time(topics_over_time_zeroshot)

# Apply log scale to the y-axis
fig.update_yaxes(type="log")

# Creating Groups for aggregation 

In [168]:
sentence_df.head()

,Country,Session,Year,Sentence,Country Name,Topic,Embedding
0,BRB,73,2018,Let me begin by congratulating Ms. María Ferna...,Barbados,47,"[-0.036270916, -0.0026696608, 0.0026795529, -0..."
1,BRB,73,2018,"However, I would like to pause at this stage, ...",Barbados,-1,"[0.015536962, 0.029011553, 0.023046829, -0.011..."
2,BRB,73,2018,Those events include the transit of a tropical...,Barbados,-1,"[0.067203976, 0.04081179, 0.11460312, 0.070455..."
3,BRB,73,2018,Those events are of great concern because the ...,Barbados,869,"[0.060707532, 0.074981414, 0.02815113, 0.03468..."
4,BRB,73,2018,"I ask myself, what does all that matter? Last ...",Barbados,-1,"[0.09178345, 0.048956443, 0.018212734, 0.02262..."


In [51]:
# Define topic-to-group mapping by topic numbers
topic_groups = {
    "Military Technology": [0, 1, 2],  # Nuclear Weapons, Biological Weapons, Chemical Weapons
    "Dual Use Technology": [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 21, 22, 23, 24],  # AI, Quantum Tech, etc.
    "Civilian Technology": [13, 14, 15, 16, 17],  # Climate change, Electric cars, etc.
    "Capacity Building": [18, 19, 20],  # Digital Divide, Tech Capacity Building, etc.
    "Grey Zone": [21, 22, 23, 24]  # Cyberattacks, Election Interference, etc.
}

# Function to assign group based on topic number(s)
def assign_group_by_number(topics):
    # If 'topics' is a single integer, convert it into a list
    if isinstance(topics, int):
        topics = [topics]  # Make it iterable

    # Check which group the topics belong to
    for group, topic_list in topic_groups.items():
        if any(topic in topic_list for topic in topics):
            return group
    return "Uncategorized"  # Default if no match found

# Apply function to create "group" column based on topic numbers
sentence_df["group"] = sentence_df["Topic"].apply(assign_group_by_number)



In [52]:
# Filter DataFrames based on the group
df_military = sentence_df[sentence_df["group"] == "Military Technology"]
df_dual_use = sentence_df[sentence_df["group"] == "Dual Use Technology"]
df_civilian = sentence_df[sentence_df["group"] == "Civilian Technology"]
df_capacity_building = sentence_df[sentence_df["group"] == "Capacity Building"]
df_grey_zone = sentence_df[sentence_df["group"] == "Grey Zone"]

# Saving the DataFrames as CSV files
df_military.to_csv('df_military.csv', index=False)
df_dual_use.to_csv('df_dual_use.csv', index=False)
df_civilian.to_csv('df_civilian.csv', index=False)
df_capacity_building.to_csv('df_capacity_building.csv', index=False)
df_grey_zone.to_csv('df_grey_zone.csv', index=False)



In [182]:
df_military

,Country,Session,Year,Sentence,Country Name,Topic,Embedding,group,bloc
48,BRB,73,2018,"I repeat, it led to two world wars and an atom...",Barbados,0,"[-0.01074151, 0.08483641, -0.014866305, 0.0476...",Military Technology,Other
374,SWE,73,2018,We are witnessing a deeply worrisome nuclear r...,Sweden,0,"[0.029459324, 0.041300286, -0.034867812, 0.035...",Military Technology,Other
376,SWE,73,2018,The framework of the Treaty on the Non-Prolife...,Sweden,0,"[0.005601457, 0.04289692, -0.0617467, -0.04264...",Military Technology,Other
1050,ALB,73,2018,We support all the actions taken by the intern...,Albania,2,"[-0.011582497, 0.044264536, 0.013668734, -0.06...",Military Technology,Other
1210,BOL,73,2018,"The second major threat is the arms race, the ...","Bolivia, Plurinational State of",0,"[0.048457425, 0.045786474, -0.022934327, 0.061...",Military Technology,Other
...,...,...,...,...,...,...,...,...,...
372979,TUR,69,2014,"If 2,000 people are killed by chemical weapons...",Türkiye,2,"[0.10385505, -0.017613554, -0.01746026, -0.005...",Military Technology,Other
372982,TUR,69,2014,Let it be understood that the use of any kind...,Türkiye,2,"[0.080962926, 0.07141191, -0.031963583, -0.004...",Military Technology,Other
374213,LUX,69,2014,The proliferation of weapons of mass destruct...,Luxembourg,0,"[-0.013056871, -0.0032054023, 0.026966093, 0.0...",Military Technology,Other
374285,TKM,69,2014,We call for consistent strengthening of intern...,Turkmenistan,0,"[0.0061750985, 0.03824251, 0.01598714, -0.0742...",Military Technology,Other


## Cosine Similarity - All years
Starting with miltiary technology creating cosine similarity for every value 

## First make a rolling average chart for every year

In [133]:
# Group countries into different blocs using three-letter country codes
country_blocs = {
    "Original EU": [
        "BEL", "FRA", "DEU", "ITA", "LUX", "NLD"
    ],
    "Total EU": [
        "AUT", "BEL", "BGR", "HRV", "CYP", "CZE", "DNK", "EST", "FIN", "FRA", "DEU", "GRC", "HUN", "IRL", "ITA", "LVA", "LTU", "LUX", 
        "MLT", "NLD", "POL", "PRT", "ROU", "SVK", "SVN", "ESP", "SWE"
    ],
    "China": ["CHN"],
    "US": ["USA"],
    "UK": ["GBR"],
    "ASEAN": [
        "BRN", "KHM", "IDN", "LAO", "MYS", "MMR", "PHL", "SGP", "THA", "VNM"
    ],
    "GRULAC": [
        "ARG", "BOL", "BRA", "CHL", "COL", "CRI", "CUB", "DOM", "ECU", "SLV", "GTM", "HND", "MEX", "NIC", "PAN", "PRY", "PER", "URY", "VEN"
    ],
    "African Union": [
        "DZA", "AGO", "BEN", "BWA", "BFA", "BDI", "CPV", "CMR", "CAF", "TCD", "COM", "COG", "COD", "DJI", "EGY", "GNQ", "ERI", "SWZ", "ETH", 
        "GAB", "GMB", "GHA", "GIN", "GNB", "CIV", "KEN", "LSO", "LBR", "LBY", "MDG", "MWI", "MLI", "MRT", "MUS", "MAR", "MOZ", "NAM", "NER", 
        "NGA", "RWA", "STP", "SEN", "SYC", "SLE", "SOM", "ZAF", "SSD", "SDN", "TGO", "TUN", "UGA", "ZMB", "ZWE"
    ],
    "Aligned Indo-Pacific": [
        "AUS", "NZL", "JPN", "KOR"
    ],
    "Non-Aligned Movement (NAM)": [
        "AFG", "BHR", "IRN", "IRQ", "JOR", "KAZ", "KWT", "KGZ", "LBN", "MDV", "MNG", "PRK", "OMN", "PSE", "QAT", "TJK", "TKM", 
        "ARE", "UZB", "YEM", "BGD", "BTN", "NPL", "LKA", "AZE", "BLR","IND"
    ]
}

def assign_bloc(country_code):
    for bloc, country_codes in country_blocs.items():
        if country_code in country_codes:
            return bloc
    return "Other"  

# Define the function to process each dataframe
def process_df(df):
    # Assign bloc based on country codes
    df['bloc'] = df['Country'].apply(assign_bloc)

    # Create a rolling year range (-2 to +2)
    df['New Year'] = df['Year']
    year_range = list(range(-1, 1))  # [-2, -1, 0, +1, +2]
    dfs = []

    for offset in year_range:
        df_copy = df.copy()
        df_copy['New Year'] = df_copy['Year'] + offset
        dfs.append(df_copy)

    # Concatenate expanded dataframes
    df_expanded = pd.concat(dfs, ignore_index=True)

    # Group by "bloc" and "New Year" to compute average embeddings
    df_grouped = df_expanded.groupby(['New Year', 'Country Name'], as_index=False)['Embedding'].apply(
        lambda x: np.mean(np.vstack(x), axis=0)
    ).reset_index()



    return df_grouped

# Apply the function to each dataframe
df_military_processed = process_df(df_military)
df_dual_use_processed = process_df(df_dual_use)
df_civilian_processed = process_df(df_civilian)
df_capacity_building_processed = process_df(df_capacity_building)
df_grey_zone_processed = process_df(df_grey_zone)


In [65]:
df_military_processed

,index,New Year,Country Name,Embedding
0,0,1990,Algeria,"[-0.003917437, 0.09966035, 0.005318463, -0.010..."
1,1,1990,Argentina,"[-0.00044626743, 0.019327445, -0.013796249, -0..."
2,2,1990,Armenia,"[0.028329864, 0.0015949451, -0.08406625, -0.05..."
3,3,1990,Australia,"[-0.018374214, 0.042369448, -0.0011329686, 0.0..."
4,4,1990,Azerbaijan,"[0.024623286, 0.051959738, -0.034831118, 0.006..."
...,...,...,...,...
4598,4598,2025,Trinidad and Tobago,"[0.022764148, 0.05206531, 0.025202315, 0.03340..."
4599,4599,2025,Ukraine,"[0.027233655, 0.075536884, 0.0033564686, 0.018..."
4600,4600,2025,United Arab Emirates,"[-0.013928543, 0.04431373, -0.008468181, 0.027..."
4601,4601,2025,United States,"[0.017288074, 0.067009196, 0.0018983912, -0.00..."


In [64]:
df_dual_use_processed.to_csv("dual_use_processed.csv", index=False)
df_military_processed.to_csv("military_processed.csv", index=False)
df_civilian_processed.to_csv("civilian_processed.csv", index=False)

## Plot for Dual-Use  Technoogy 

In [ ]:


base_number = 2021
# List of countries to compare against
countries_of_interest = ["United States", "United Kingdom", "China", "Germany"]

def calculate_cosine_with_base_year(df, countries_of_interest, base_year=base_number, embeddings_column='Embedding'):
    similarity_matrix = []

    # Remove the years 1990, 1991, and 2024
    df = df[~df['New Year'].isin([1990, 1991, 2024, 2025])]

    # Extract the embeddings for the base year (2008) for the countries of interest
    base_year_data = df[df['New Year'] == base_year]
    base_embeddings = {
        country: base_year_data[base_year_data['Country Name'] == country][embeddings_column].values.tolist()
        for country in countries_of_interest
    }

    # Iterate over each year in the "New Year" column
    for year in df['New Year'].unique():
        year_data = df[df['New Year'] == year]
        year_matrix = {'New Year': year}

        # Get embeddings for all countries in the given year
        year_embeddings = {
            country: year_data[year_data['Country Name'] == country][embeddings_column].values.tolist()
            for country in year_data['Country Name'].unique()
        }

        # Compare the base year (2008) for each country of interest with all other countries
        for country in countries_of_interest:
            if len(base_embeddings[country]) > 0:
                for other_country, other_country_embedding in year_embeddings.items():
                    if len(other_country_embedding) > 0:
                        similarity = cosine_similarity([base_embeddings[country][0]], [other_country_embedding[0]])
                        year_matrix[f'Cosine Similarity {country} vs {other_country}'] = similarity[0][0]
                    else:
                        # If no data for the other country in the given year, assign None
                        year_matrix[f'Cosine Similarity {country} vs {other_country}'] = None
            else:
                # If no data for the country in the base year, assign None
                year_matrix[f'Cosine Similarity {country}'] = None

        similarity_matrix.append(year_matrix)

    similarity_df = pd.DataFrame(similarity_matrix)

    # Drop columns where more than 40% of the values are NaN
    threshold = len(similarity_df) * 0.4  # 40% of the rows
    similarity_df = similarity_df.dropna(thresh=threshold, axis=1)

    # Ensure that only columns with less than 40% NaN values are kept
    return similarity_df

# Compute the cosine similarity with the base year for each dataset
cosine_sim_dual_use = calculate_cosine_with_base_year(df_dual_use_processed, countries_of_interest)


df_list = [cosine_sim_dual_use]  # You can add cosine_sim_military, cosine_sim_civilian as needed
df_names = ["Dual Use"]  # "Military", "Civilian" as needed

# Loop through each DataFrame (Dual Use, Military, Civilian)
for cosine_sim_df, name in zip(df_list, df_names):
    for reference_country in countries_of_interest:  # The countries of interest (e.g., United States)
        # Step 1: Filter for columns containing the relevant cosine similarity for each country
        filtered_df = cosine_sim_df.filter(like=f"Cosine Similarity {reference_country} vs", axis=1)

        # Step 2: Keep 'New Year' and rename columns to remove redundant text
        melted_df = cosine_sim_df[['New Year']].join(filtered_df)
        melted_df.columns = melted_df.columns.str.replace(f"Cosine Similarity {reference_country} vs ", "")

        # Step 3: Convert to long format for Plotly
        melted_df = melted_df.melt(id_vars=['New Year'], var_name='Country', value_name='Similarity')

        # Step 4: Convert NaN to None for proper Plotly rendering
        melted_df['Similarity'] = melted_df['Similarity'].where(melted_df['Similarity'].notna(), None)

        # Step 5: Extract the last row for each country to add labels at the end
        last_row = melted_df.groupby('Country').tail(1)

        # Step 6: Create the line plot with Plotly's graph_objects for more flexibility
        fig = go.Figure()

        # Add the lines for each country
        for country in melted_df['Country'].unique():
            country_data = melted_df[melted_df['Country'] == country]
            fig.add_trace(go.Scatter(
                x=country_data['New Year'],
                y=country_data['Similarity'],
                mode='lines+markers',
                name=country,
                line=dict(shape='spline'),
                connectgaps=True,  # Connect gaps where NaN values are present
                visible='legendonly')
            )

        # Step 7: Update the layout
        fig.update_layout(
            title=f'Cosine Similarity of Each Country with {reference_country} in {base_number} ({name} Data)',
            xaxis_title='Year',
            yaxis_title='Cosine Similarity',
            yaxis=dict(range=[0, 1]),
            showlegend=True,
            hovermode='closest')
        

        # Show the plot
        fig.show()


## Plot for military technology 

In [173]:


base_number = 2021
# List of countries to compare against
countries_of_interest = ["United States", "United Kingdom", "China", "Germany"]

def calculate_cosine_with_base_year(df, countries_of_interest, base_year=base_number, embeddings_column='Embedding'):
    similarity_matrix = []

    # Remove the years 1990, 1991, and 2024
    df = df[~df['New Year'].isin([1990, 1991, 2024, 2025])]

    # Extract the embeddings for the base year (2008) for the countries of interest
    base_year_data = df[df['New Year'] == base_year]
    base_embeddings = {
        country: base_year_data[base_year_data['Country Name'] == country][embeddings_column].values.tolist()
        for country in countries_of_interest
    }

    # Iterate over each year in the "New Year" column
    for year in df['New Year'].unique():
        year_data = df[df['New Year'] == year]
        year_matrix = {'New Year': year}

        # Get embeddings for all countries in the given year
        year_embeddings = {
            country: year_data[year_data['Country Name'] == country][embeddings_column].values.tolist()
            for country in year_data['Country Name'].unique()
        }

        # Compare the base year (2008) for each country of interest with all other countries
        for country in countries_of_interest:
            if len(base_embeddings[country]) > 0:
                for other_country, other_country_embedding in year_embeddings.items():
                    if len(other_country_embedding) > 0:
                        similarity = cosine_similarity([base_embeddings[country][0]], [other_country_embedding[0]])
                        year_matrix[f'Cosine Similarity {country} vs {other_country}'] = similarity[0][0]
                    else:
                        # If no data for the other country in the given year, assign None
                        year_matrix[f'Cosine Similarity {country} vs {other_country}'] = None
            else:
                # If no data for the country in the base year, assign None
                year_matrix[f'Cosine Similarity {country}'] = None

        similarity_matrix.append(year_matrix)

    similarity_df = pd.DataFrame(similarity_matrix)

    # Drop columns where more than 40% of the values are NaN
    threshold = len(similarity_df) * 0.4  # 40% of the rows
    similarity_df = similarity_df.dropna(thresh=threshold, axis=1)

    # Ensure that only columns with less than 40% NaN values are kept
    return similarity_df

# Compute the cosine similarity with the base year for each dataset
cosine_sim_military = calculate_cosine_with_base_year(df_military_processed, countries_of_interest)


df_list = [cosine_sim_military]  # You can add cosine_sim_military, cosine_sim_civilian as needed
df_names = ["MIlitary"]  # "Military", "Civilian" as needed

# Loop through each DataFrame (Dual Use, Military, Civilian)
for cosine_sim_df, name in zip(df_list, df_names):
    for reference_country in countries_of_interest:  # The countries of interest (e.g., United States)
        # Step 1: Filter for columns containing the relevant cosine similarity for each country
        filtered_df = cosine_sim_df.filter(like=f"Cosine Similarity {reference_country} vs", axis=1)

        # Step 2: Keep 'New Year' and rename columns to remove redundant text
        melted_df = cosine_sim_df[['New Year']].join(filtered_df)
        melted_df.columns = melted_df.columns.str.replace(f"Cosine Similarity {reference_country} vs ", "")

        # Step 3: Convert to long format for Plotly
        melted_df = melted_df.melt(id_vars=['New Year'], var_name='Country', value_name='Similarity')

        # Step 4: Convert NaN to None for proper Plotly rendering
        melted_df['Similarity'] = melted_df['Similarity'].where(melted_df['Similarity'].notna(), None)

        # Step 5: Extract the last row for each country to add labels at the end
        last_row = melted_df.groupby('Country').tail(1)

        # Step 6: Create the line plot with Plotly's graph_objects for more flexibility
        fig = go.Figure()

        # Add the lines for each country
        for country in melted_df['Country'].unique():
            country_data = melted_df[melted_df['Country'] == country]
            fig.add_trace(go.Scatter(
                x=country_data['New Year'],
                y=country_data['Similarity'],
                mode='lines+markers',
                name=country,
                line=dict(shape='spline'),
                connectgaps=True,  # Connect gaps where NaN values are present
                visible='legendonly')
            )

        # Step 7: Update the layout
        fig.update_layout(
            title=f'Cosine Similarity of Each Country with {reference_country} in {base_number} ({name} Data)',
            xaxis_title='Year',
            yaxis_title='Cosine Similarity',
            yaxis=dict(range=[0, 1]),
            showlegend=True,
            hovermode='closest')
        

        # Show the plot
        fig.show()

## Plot for Civilian Technology 

In [177]:


base_number = 2022
# List of countries to compare against
countries_of_interest = ["United States", "United Kingdom", "China", "Germany"]

def calculate_cosine_with_base_year(df, countries_of_interest, base_year=base_number, embeddings_column='Embedding'):
    similarity_matrix = []

    # Remove the years 1990, 1991, and 2024
    df = df[~df['New Year'].isin([1990, 1991, 2024, 2025])]

    # Extract the embeddings for the base year (2008) for the countries of interest
    base_year_data = df[df['New Year'] == base_year]
    base_embeddings = {
        country: base_year_data[base_year_data['Country Name'] == country][embeddings_column].values.tolist()
        for country in countries_of_interest
    }

    # Iterate over each year in the "New Year" column
    for year in df['New Year'].unique():
        year_data = df[df['New Year'] == year]
        year_matrix = {'New Year': year}

        # Get embeddings for all countries in the given year
        year_embeddings = {
            country: year_data[year_data['Country Name'] == country][embeddings_column].values.tolist()
            for country in year_data['Country Name'].unique()
        }

        # Compare the base year (2008) for each country of interest with all other countries
        for country in countries_of_interest:
            if len(base_embeddings[country]) > 0:
                for other_country, other_country_embedding in year_embeddings.items():
                    if len(other_country_embedding) > 0:
                        similarity = cosine_similarity([base_embeddings[country][0]], [other_country_embedding[0]])
                        year_matrix[f'Cosine Similarity {country} vs {other_country}'] = similarity[0][0]
                    else:
                        # If no data for the other country in the given year, assign None
                        year_matrix[f'Cosine Similarity {country} vs {other_country}'] = None
            else:
                # If no data for the country in the base year, assign None
                year_matrix[f'Cosine Similarity {country}'] = None

        similarity_matrix.append(year_matrix)

    similarity_df = pd.DataFrame(similarity_matrix)

    # Drop columns where more than 40% of the values are NaN
    threshold = len(similarity_df) * 0.4  # 40% of the rows
    similarity_df = similarity_df.dropna(thresh=threshold, axis=1)

    # Ensure that only columns with less than 40% NaN values are kept
    return similarity_df

# Compute the cosine similarity with the base year for each dataset
cosine_sim_civilian = calculate_cosine_with_base_year(df_civilian_processed, countries_of_interest)


df_list = [cosine_sim_civilian]  # You can add cosine_sim_military, cosine_sim_civilian as needed
df_names = ["Civilian"]  # "Military", "Civilian" as needed

# Loop through each DataFrame (Dual Use, Military, Civilian)
for cosine_sim_df, name in zip(df_list, df_names):
    for reference_country in countries_of_interest:  # The countries of interest (e.g., United States)
        # Step 1: Filter for columns containing the relevant cosine similarity for each country
        filtered_df = cosine_sim_df.filter(like=f"Cosine Similarity {reference_country} vs", axis=1)

        # Step 2: Keep 'New Year' and rename columns to remove redundant text
        melted_df = cosine_sim_df[['New Year']].join(filtered_df)
        melted_df.columns = melted_df.columns.str.replace(f"Cosine Similarity {reference_country} vs ", "")

        # Step 3: Convert to long format for Plotly
        melted_df = melted_df.melt(id_vars=['New Year'], var_name='Country', value_name='Similarity')

        # Step 4: Convert NaN to None for proper Plotly rendering
        melted_df['Similarity'] = melted_df['Similarity'].where(melted_df['Similarity'].notna(), None)

        # Step 5: Extract the last row for each country to add labels at the end
        last_row = melted_df.groupby('Country').tail(1)

        # Step 6: Create the line plot with Plotly's graph_objects for more flexibility
        fig = go.Figure()

        # Add the lines for each country
        for country in melted_df['Country'].unique():
            country_data = melted_df[melted_df['Country'] == country]
            fig.add_trace(go.Scatter(
                x=country_data['New Year'],
                y=country_data['Similarity'],
                mode='lines+markers',
                name=country,
                line=dict(shape='spline'),
                connectgaps=True,  # Connect gaps where NaN values are present
                visible='legendonly')
            )

        # Step 7: Update the layout
        fig.update_layout(
            title=f'Cosine Similarity of Each Country with {reference_country} in {base_number} ({name} Data)',
            xaxis_title='Year',
            yaxis_title='Cosine Similarity',
            yaxis=dict(range=[0, 1]),
            showlegend=True,
            hovermode='closest')
        

        # Show the plot
        fig.show()

Civilian data must be taken from before COVID because "health technology" pulls everything towards 2020. 

## Showing Average Variation for Miltiary, Civlian, Dual-use, and capacity building 

In [164]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from sklearn.metrics.pairwise import cosine_similarity

def calculate_cosine_with_yearly_average(df, embeddings_column='Embedding'):
    similarity_matrix = []

    # Remove unwanted years (1990, 1991, 2024, 2025)
    df = df[~df['New Year'].isin([1990, 1991, 2024, 2025])]

    # Sort data by year to ensure correct ordering
    df = df.sort_values(by=['New Year'])

    # Iterate over each year in the dataset
    for year in df['New Year'].unique():
        year_data = df[df['New Year'] == year]

        # Compute the average embedding for the year
        all_embeddings = np.stack(year_data[embeddings_column].dropna().values)  # Drop NaN and convert to array
        if len(all_embeddings) == 0:
            continue  # Skip if no embeddings exist for the year
        
        avg_embedding = np.mean(all_embeddings, axis=0)  # Compute mean embedding for the year

        year_matrix = {'New Year': year}

        for country in year_data['Country Name'].unique():
            current_embedding = year_data[year_data['Country Name'] == country][embeddings_column].values

            if len(current_embedding) > 0:
                # Compute cosine similarity between the country’s embedding and the yearly average
                similarity = cosine_similarity([current_embedding[0]], [avg_embedding])[0][0]
                year_matrix[f'Cosine Similarity {country}'] = similarity
            else:
                year_matrix[f'Cosine Similarity {country}'] = None  # No embedding available

        similarity_matrix.append(year_matrix)

    similarity_df = pd.DataFrame(similarity_matrix)

    # Drop columns where more than 40% of the values are NaN
    threshold = len(similarity_df) * 0.4  # 40% of rows
    similarity_df = similarity_df.dropna(thresh=threshold, axis=1)

    return similarity_df

# Compute cosine similarity with yearly average for each dataset
cosine_sim_military = calculate_cosine_with_yearly_average(df_military_processed)
cosine_sim_civilian = calculate_cosine_with_yearly_average(df_civilian_processed)
cosine_sim_dual_use = calculate_cosine_with_yearly_average(df_dual_use_processed)

# DataFrames (cosine similarity results) to process
df_list = [cosine_sim_military, cosine_sim_civilian, cosine_sim_dual_use]  # Order: Military, Civilian, Dual-Use
df_names = ["Military", "Civilian", "Dual-Use"]  # Corresponding dataset names

# Loop through each DataFrame (e.g., Military, Civilian, Dual-Use)
for cosine_sim_df, name in zip(df_list, df_names):
    # Create a new figure for each category
    fig = go.Figure()
    
    # Step 1: Rename columns to remove "Cosine Similarity" prefix
    renamed_df = cosine_sim_df.rename(columns=lambda col: col.replace("Cosine Similarity ", "") if "Cosine Similarity" in col else col)

    # Step 2: Convert the DataFrame into long format for Plotly
    melted_df = renamed_df.melt(id_vars=['New Year'], var_name='Country', value_name='Similarity')

    # Step 3: Convert NaN to None for proper Plotly rendering
    melted_df['Similarity'] = melted_df['Similarity'].where(melted_df['Similarity'].notna(), None)

    # Step 4: Add traces for each country
    for country in melted_df['Country'].unique():
        country_data = melted_df[melted_df['Country'] == country]
        fig.add_trace(go.Scatter(
            x=country_data['New Year'],
            y=country_data['Similarity'],
            mode='lines+markers',
            name=country,  # Show only the country name
            line=dict(shape='spline'),
            connectgaps=True,  # Connect gaps where NaN values are present
            visible='legendonly'  # Deselect all by default
        ))

    # Step 5: Update layout
    fig.update_layout(
        title=f'Cosine Similarity of Countries Over Time ({name})',
        xaxis_title='Year',
        yaxis_title='Cosine Similarity',
        yaxis=dict(range=[0, 1]),
        showlegend=True,
        hovermode='closest'
    )

    # Show the plot
    fig.show()




So for civilian, because vaccines are part of health technology and people talk a lot more about vaccines during COVID than they have in the past then the values are very close to the "average" this is because the average has such a large number of responses to do with COVID it means that the average is weighted towards that year. It does not tell us much about general patterns. 

## PCA Analysis 

In [ ]:
# Load GDP data, skipping first four rows to get correct column names
gdp_df = pd.read_csv("GDP.csv", skiprows=4)

gdp_df = gdp_df.drop(columns=['Unnamed: 68'])

# Convert wide format to long format
gdp_long = gdp_df.melt(id_vars=["Country Name", "Country Code", "Indicator Name", "Indicator Code"],
                        var_name="Year", value_name="GDP")

# Convert Year column to integer
gdp_long["Year"] = gdp_long["Year"].astype(int)

# Merge with sentences_df on country code and year
merged_df = sentence_df.merge(gdp_long, left_on=["Country", "Year"], right_on=["Country Code", "Year"], how="left")

# Display results
merged_df.head()

merged_df = merged_df.drop(columns=['Country Name_y', 'Country Code', "Indicator Name", 'Indicator Code'])

merged = merged_df.rename(columns={'Country Name_x': 'Country Name'})


In [189]:
merged_df = merged_df.rename(columns={'Country Name_x': 'Country Name'})
merged_df

,Country,Session,Year,Sentence,Country Name,Topic,Embedding,group,GDP
0,BRB,73,2018,Let me begin by congratulating Ms. María Ferna...,Barbados,95,"[-0.036270916, -0.0026696608, 0.0026795529, -0...",Uncategorized,5.188858e+09
1,BRB,73,2018,"However, I would like to pause at this stage, ...",Barbados,-1,"[0.015536962, 0.029011553, 0.023046829, -0.011...",Uncategorized,5.188858e+09
2,BRB,73,2018,Those events include the transit of a tropical...,Barbados,-1,"[0.067203976, 0.04081179, 0.11460312, 0.070455...",Uncategorized,5.188858e+09
3,BRB,73,2018,Those events are of great concern because the ...,Barbados,-1,"[0.060707532, 0.074981414, 0.02815113, 0.03468...",Uncategorized,5.188858e+09
4,BRB,73,2018,"I ask myself, what does all that matter? Last ...",Barbados,1166,"[0.09178345, 0.048956443, 0.018212734, 0.02262...",Uncategorized,5.188858e+09
...,...,...,...,...,...,...,...,...,...
522324,HUN,69,2014,"We have to heed the advice of Nicholas Stern, ...",Hungary,1132,"[0.056570105, 0.06454281, 0.05719191, 0.002399...",Uncategorized,2.977526e+11
522325,HUN,69,2014,I have just cited the example of New York City,Hungary,946,"[0.12445413, -0.03829658, 0.05360141, 0.039285...",Uncategorized,2.977526e+11
522326,HUN,69,2014,A few hundred million dollars would have been ...,Hungary,-1,"[0.014341723, 0.11719479, 0.060230006, -0.0165...",Uncategorized,2.977526e+11
522327,HUN,69,2014,The New York example tells us that we are not ...,Hungary,-1,"[0.10041228, 0.006510497, 0.10341268, 0.040018...",Uncategorized,2.977526e+11


In [190]:
# Filter DataFrames based on the group from the merged DataFrame
PCA_military = merged_df[merged_df["group"] == "Military Technology"]
PCA_dual_use = merged_df[merged_df["group"] == "Dual Use Technology"]
PCA_civilian = merged_df[merged_df["group"] == "Civilian Technology"]
PCA_capacity_building = merged_df[merged_df["group"] == "Capacity Building"]
PCA_grey_zone = merged_df[merged_df["group"] == "Grey Zone"]

# Saving the DataFrames with specific names for each group
PCA_military.to_csv('PCA_military.csv', index=False)
PCA_dual_use.to_csv('PCA_dual_use.csv', index=False)
PCA_civilian.to_csv('PCA_civilian.csv', index=False)
PCA_capacity_building.to_csv('PCA_capacity_building.csv', index=False)
PCA_grey_zone.to_csv('PCA_grey_zone.csv', index=False)


/Users/student/anaconda3/envs/Capstone/lib/python3.11/site-packages/pandas/core/indexes/base.py:7834: RuntimeWarning:

invalid value encountered in cast



In [191]:
PCA_military 

,Country,Session,Year,Sentence,Country Name,Topic,Embedding,group,GDP
48,BRB,73,2018,"I repeat, it led to two world wars and an atom...",Barbados,0,"[-0.01074151, 0.08483641, -0.014866305, 0.0476...",Military Technology,5.188858e+09
374,SWE,73,2018,We are witnessing a deeply worrisome nuclear r...,Sweden,0,"[0.029459324, 0.041300286, -0.034867812, 0.035...",Military Technology,6.131991e+11
376,SWE,73,2018,The framework of the Treaty on the Non-Prolife...,Sweden,0,"[0.005601457, 0.04289692, -0.0617467, -0.04264...",Military Technology,6.131991e+11
1050,ALB,73,2018,We support all the actions taken by the intern...,Albania,2,"[-0.011582497, 0.044264536, 0.013668734, -0.06...",Military Technology,4.213154e+10
1210,BOL,73,2018,"The second major threat is the arms race, the ...","Bolivia, Plurinational State of",0,"[0.048457425, 0.045786474, -0.022934327, 0.061...",Military Technology,1.140141e+11
...,...,...,...,...,...,...,...,...,...
520349,TUR,69,2014,"If 2,000 people are killed by chemical weapons...",Türkiye,2,"[0.10385505, -0.017613554, -0.01746026, -0.005...",Military Technology,1.896628e+12
520352,TUR,69,2014,Let it be understood that the use of any kind...,Türkiye,2,"[0.080962926, 0.07141191, -0.031963583, -0.004...",Military Technology,1.896628e+12
521583,LUX,69,2014,The proliferation of weapons of mass destruct...,Luxembourg,0,"[-0.013056871, -0.0032054023, 0.026966093, 0.0...",Military Technology,7.201170e+10
521655,TKM,69,2014,We call for consistent strengthening of intern...,Turkmenistan,0,"[0.0061750985, 0.03824251, 0.01598714, -0.0742...",Military Technology,7.621077e+10


In [192]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler


In [208]:


# 1. Group the data by 5-year periods starting from 1994
PCA_military['Period'] = ((PCA_military['Year'] - 1994) // 5) * 5 + 1994  # Grouping by 5-year periods

# 2. Calculate the average GDP for each country in each period, excluding NaN values
gdp_df = PCA_military.groupby(['Country Name', 'Period'])['GDP'].apply(lambda x: x.dropna().mean()).reset_index()

# 3. Calculate the average embedding, excluding NaN values
embeddings_df = PCA_military.groupby(['Country Name', 'Period'])['Embedding'].apply(lambda x: np.mean(np.vstack(x[~x.isna()]), axis=0)).reset_index()

# 4. Merge embeddings and GDP data on Country and Period
merged_embeddings_gdp = pd.merge(embeddings_df, gdp_df, on=['Country Name', 'Period'])

# Drop any rows with NaN GDP after the merge
merged_embeddings_gdp = merged_embeddings_gdp.dropna(subset=['GDP'])

# 5. Grouped data by Country and Period for average embeddings and GDP
grouped_data = merged_embeddings_gdp.groupby(['Country Name', 'Period']).agg(
    avg_embedding=('Embedding', 'mean'),
    avg_gdp=('GDP', 'mean')
).reset_index()

# 6. Apply standard scaling on the embeddings before PCA
scaler = StandardScaler()
scaled_embeddings = scaler.fit_transform(np.vstack(grouped_data['avg_embedding']))

# 7. Perform PCA on the embeddings
pca = PCA(n_components=2)  # Reduce to 2 components for visualization
pca_result = pca.fit_transform(scaled_embeddings)

# 8. Add PCA results back to the dataframe
grouped_data['PCA1'] = pca_result[:, 0]
grouped_data['PCA2'] = pca_result[:, 1]

sorted_periods = sorted(grouped_data['Period'].unique())  # Sort periods in ascending order

# 10. Plotting for each period (creating separate plots)
for period in sorted_periods:
    period_data = grouped_data[grouped_data['Period'] == period]
    
    # Prepare the plot for the current period
    fig = go.Figure()
    
    # Add scatter plots for each country with the size of the dot corresponding to average GDP
    for country in period_data['Country Name'].unique():
        country_data = period_data[period_data['Country Name'] == country]
        fig.add_trace(go.Scatter(
            x=country_data['PCA1'],
            y=country_data['PCA2'],
            mode='markers',
            name=country,
            marker=dict(
                size=country_data['avg_gdp'] / 100000000000,  # Scale size for visualization
                opacity=0.6,
                line=dict(width=1, color='black')
            ),
            #visible='legendonly'  # Deselect all by default
        ))

    # Update layout with labels and title for each period
    fig.update_layout(
        title=f"PCA of Embeddings for Period {period} with GDP Size",
        xaxis_title='PCA 1',
        yaxis_title='PCA 2',
        showlegend=True,
        hovermode='closest'
    )

    # Show the plot for the current period
    fig.show()


In [221]:
from sklearn.preprocessing import MinMaxScaler



In [222]:


# 1. Group the data by 5-year periods starting from 1994
PCA_military['Period'] = ((PCA_military['Year'] - 1994) // 5) * 5 + 1994  # Grouping by 5-year periods

# 2. Calculate the average GDP for each country in each period, excluding NaN values
gdp_df = PCA_military.groupby(['Country Name', 'Period'])['GDP'].apply(lambda x: x.dropna().mean()).reset_index()

scaler = MinMaxScaler()

# Scale the GDP values and add to a new column 'scaled_gdp'
gdp_df['scaled_gdp'] = scaler.fit_transform(gdp_df[['GDP']])


# 3. Calculate the average embedding, excluding NaN values
embeddings_df = PCA_military.groupby(['Country Name', 'Period'])['Embedding'].apply(lambda x: np.mean(np.vstack(x[~x.isna()]), axis=0)).reset_index()

# 4. Merge embeddings and GDP data on Country and Period
merged_embeddings_gdp = pd.merge(embeddings_df, gdp_df, on=['Country Name', 'Period'])

# Drop any rows with NaN GDP after the merge
merged_embeddings_gdp = merged_embeddings_gdp.dropna(subset=['GDP'])

# 5. Grouped data by Country and Period for average embeddings and GDP
grouped_data = merged_embeddings_gdp.groupby(['Country Name', 'Period']).agg(
    avg_embedding=('Embedding', 'mean'),
    avg_gdp=('GDP', 'mean')
).reset_index()

# 6. Apply standard scaling on the embeddings before PCA
scaler = StandardScaler()
scaled_embeddings = scaler.fit_transform(np.vstack(grouped_data['avg_embedding']))

# 7. Perform PCA on the embeddings
pca = PCA(n_components=2)  # Reduce to 2 components for visualization
pca_result = pca.fit_transform(scaled_embeddings)

# 8. Add PCA results back to the dataframe
grouped_data['PCA1'] = pca_result[:, 0]
grouped_data['PCA2'] = pca_result[:, 1]

# 9. Perform K-Means clustering on the PCA results
kmeans = KMeans(n_clusters=4, random_state=42)  # You can adjust the number of clusters
grouped_data['Cluster'] = kmeans.fit_predict(grouped_data[['PCA1', 'PCA2']])

# 10. Sort periods from oldest to newest
sorted_periods = sorted(grouped_data['Period'].unique())  # Sort periods in ascending order

# 11. Plotting for each period (creating separate plots)
for period in sorted_periods:
    period_data = grouped_data[grouped_data['Period'] == period]
    
    # Prepare the plot for the current period
    fig = go.Figure()
    
    # Add scatter plots for each country with the size of the dot corresponding to average GDP
    fig.add_trace(go.Scatter(
        x=period_data['PCA1'],
        y=period_data['PCA2'],
        mode='markers',
        name=f'Period {period}',
        marker=dict(
            size=period_data['avg_gdp'] / 100000000000,  # Scale size for visualization
            opacity=0.6,
            line=dict(width=1, color='black'),
            color=period_data['Cluster'],  # Color points by cluster
            colorscale='Viridis',  # Color scale for the clusters
            colorbar=dict(title='Cluster')  # Color bar for clusters
        ),
        text=period_data['Country Name'],  # Show country name on hover
    ))

    # Update layout with labels and title for each period
    fig.update_layout(
        title=f"PCA of Embeddings for Period {period} with GDP Size and K-Means Clusters",
        xaxis_title='PCA 1',
        yaxis_title='PCA 2',
        showlegend=True,
        hovermode='closest'
    )

    # Show the plot for the current period
    fig.show()


In [213]:
from sklearn.cluster import DBSCAN
import plotly.graph_objects as go
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Group the data by 5-year periods starting from 1994
PCA_military['Period'] = ((PCA_military['Year'] - 1994) // 5) * 5 + 1994  # Grouping by 5-year periods

# Calculate the average GDP for each country in each period, excluding NaN values
gdp_df = PCA_military.groupby(['Country Name', 'Period'])['GDP'].apply(lambda x: x.dropna().mean()).reset_index()

# Calculate the average embedding, excluding NaN values
embeddings_df = PCA_military.groupby(['Country Name', 'Period'])['Embedding'].apply(lambda x: np.mean(np.vstack(x[~x.isna()]), axis=0)).reset_index()

# Merge embeddings and GDP data on Country and Period
merged_embeddings_gdp = pd.merge(embeddings_df, gdp_df, on=['Country Name', 'Period'])

# Drop any rows with NaN GDP after the merge
merged_embeddings_gdp = merged_embeddings_gdp.dropna(subset=['GDP'])

# Grouped data by Country and Period for average embeddings and GDP
grouped_data = merged_embeddings_gdp.groupby(['Country Name', 'Period']).agg(
    avg_embedding=('Embedding', 'mean'),
    avg_gdp=('GDP', 'mean')
).reset_index()

# Apply standard scaling on the embeddings before PCA
scaler = StandardScaler()
scaled_embeddings = scaler.fit_transform(np.vstack(grouped_data['avg_embedding']))

# Perform PCA on the embeddings
pca = PCA(n_components=2)  # Reduce to 2 components for visualization
pca_result = pca.fit_transform(scaled_embeddings)

# Add PCA results back to the dataframe
grouped_data['PCA1'] = pca_result[:, 0]
grouped_data['PCA2'] = pca_result[:, 1]

# Perform DBSCAN clustering on the PCA results
dbscan = DBSCAN(eps=0.5, min_samples=5)  # Parameters can be adjusted
grouped_data['Cluster'] = dbscan.fit_predict(grouped_data[['PCA1', 'PCA2']])

# Sort periods from oldest to newest
sorted_periods = sorted(grouped_data['Period'].unique())  # Sort periods in ascending order

# Plotting for each period (creating separate plots)
for period in sorted_periods:
    period_data = grouped_data[grouped_data['Period'] == period]
    
    # Prepare the plot for the current period
    fig = go.Figure()
    
    # Add scatter plots for each country with the size of the dot corresponding to average GDP
    fig.add_trace(go.Scatter(
        x=period_data['PCA1'],
        y=period_data['PCA2'],
        mode='markers',
        name=f'Period {period}',
        marker=dict(
            size=period_data['avg_gdp'] / 100000000000,  # Scale size for visualization
            opacity=0.6,
            line=dict(width=1, color='black'),
            color=period_data['Cluster'],  # Color points by cluster
            colorscale='Viridis',  # Color scale for the clusters
            colorbar=dict(title='Cluster')  # Color bar for clusters
        ),
        text=period_data['Country Name'],  # Show country name on hover
    ))

    # Update layout with labels and title for each period
    fig.update_layout(
        title=f"PCA of Embeddings for Period {period} with GDP Size and DBSCAN Clusters",
        xaxis_title='PCA 1',
        yaxis_title='PCA 2',
        showlegend=True,
        hovermode='closest'
    )

    # Show the plot for the current period
    fig.show()


In [197]:
# Display the rows where data is missing in any column
missing_data_rows = merged_embeddings_gdp[merged_embeddings_gdp.isnull().any(axis=1)]
missing_data_rows


,Country Name,Period,Embedding,GDP
0,Afghanistan,1994,"[0.036915474, 0.070349604, 0.031502873, 0.0217...",NaN
184,Cuba,1994,"[-0.055764496, 0.037322585, 0.04110789, 0.0002...",NaN
185,Cuba,1999,"[0.025159057, 0.04103466, 0.02013264, -0.00956...",NaN
186,Cuba,2004,"[0.014553455, 0.07134506, 0.013625699, 0.00568...",NaN
187,Cuba,2009,"[-0.011933499, 0.070004076, -0.0075313207, 0.0...",NaN
188,Cuba,2014,"[-0.038152855, 0.102020584, -0.023920141, 0.00...",NaN
189,Cuba,2019,"[-0.0021252837, 0.029470934, -0.025233084, 0.0...",NaN
211,Djibouti,1994,"[-0.042144902, 0.048103444, -0.009869823, -0.0...",NaN
319,Holy See (Vatican City State),2004,"[0.0016307281, 0.08461153, 0.00057803304, 0.00...",NaN
320,Holy See (Vatican City State),2009,"[-0.015786372, 0.09805957, -0.028198559, 0.010...",NaN


In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import plotly.graph_objects as go
import pandas as pd
import numpy as np

# Drop rows with NaN values in the GDP column
PCA_military_clean = PCA_military.dropna(subset=['GDP'])

# Create a new dataframe for the average embeddings and GDP over the 5-year periods
PCA_military_clean['Year Period'] = ((PCA_military_clean['Year'] - 1994) // 5) * 5 + 1994

# Group by country and year period to get the average embedding for each group
grouped_data = PCA_military_clean.groupby(['Country Name', 'Year Period']).agg(
    avg_embedding=('Embedding', 'mean'),
    avg_gdp=('GDP', 'mean')
).reset_index()

# Apply standard scaling on the embeddings before PCA
scaler = StandardScaler()
scaled_embeddings = scaler.fit_transform(np.vstack(grouped_data['avg_embedding']))

# Perform PCA on the embeddings
pca = PCA(n_components=2)  # Reduce to 2 components for visualization
pca_result = pca.fit_transform(scaled_embeddings)

# Add PCA results back to the dataframe
grouped_data['PCA1'] = pca_result[:, 0]
grouped_data['PCA2'] = pca_result[:, 1]

# Prepare the plot
fig = go.Figure()

# Add scatter plots for each country with the size of the dot corresponding to average GDP
for country in grouped_data['Country Name'].unique():
    country_data = grouped_data[grouped_data['Country Name'] == country]
    fig.add_trace(go.Scatter(
        x=country_data['PCA1'],
        y=country_data['PCA2'],
        mode='markers',
        name=country,
        marker=dict(
            size=country_data['avg_gdp'] / 1000,  # Scale size for visualization
            opacity=0.6,
            line=dict(width=1, color='black')
        ),
        visible='legendonly'  # Deselect all by default
    ))

# Update layout with labels and title
fig.update_layout(
    title="PCA of Embeddings by Country with GDP Size",
    xaxis_title='PCA 1',
    yaxis_title='PCA 2',
    showlegend=True,
    hovermode='closest'
)

# Show the plot
fig.show()


In [ ]:
# 4. Perform PCA on the embeddings
scaler = StandardScaler()
scaled_embeddings = scaler.fit_transform(np.vstack(merged_embeddings_gdp['Embedding']))

pca = PCA(n_components=2)
pca_result = pca.fit_transform(scaled_embeddings)

# Add PCA results to the dataframe
merged_embeddings_gdp['PCA1'] = pca_result[:, 0]
merged_embeddings_gdp['PCA2'] = pca_result[:, 1]

# 5. Create the PCA plot using Plotly
fig = go.Figure()

# Add traces for each country
for country in merged_embeddings_gdp['Country Name'].unique():
    country_data = merged_embeddings_gdp[merged_embeddings_gdp['Country Name'] == country]
    fig.add_trace(go.Scatter(
        x=country_data['PCA1'],
        y=country_data['PCA2'],
        mode='markers',
        name=country,
        marker=dict(
            size=country_data['GDP'] / 1000,  # Scale GDP for marker size
            opacity=0.7
        ),
        visible='legendonly'  # Hide the traces by default, can be toggled via legend
    ))

# 6. Update layout
fig.update_layout(
    title='PCA of Military Technology Embeddings (Grouped by 5-Year Periods)',
    xaxis_title='PCA Component 1',
    yaxis_title='PCA Component 2',
    showlegend=True,
    hovermode='closest',
    legend=dict(
        title="Country",
        x=1.05,  # Place legend outside the plot area
        y=1
    )
)

# Show the plot
fig.show()